# Is Google biased in Representing Gun-related Issues in the US?
TODO: complete names
Joris, Laurie, Rama, Reza, Vic

Assignment for Module 1 of NMRM Course

## Introduction
TODOs:
* Discussions in US around Biasness of Google
* The subject of gun and firearm, and how it has become a political discussion in US
* What is the well-defined claim on this biasness? Are there any claim directly about media being politically biased against this discussion?

(Will be filled from the [Google Doc](https://docs.google.com/document/d/1Vn5gixfqObaT2fnBjv8SkHH2RwmZA7VP6_rtFlOs1go/edit#))

## Methodology


In [1]:
SIMPLE_QUERIES = ("gun", "firearm", "gun control", "gun rights", "mass shooting")

### Manual Analysis of the Search Results

In [2]:
STOP = 10
from googlesearch import search

search_results = {}
for q in SIMPLE_QUERIES:
    search_results[q] = list(search(q, tld='com', lang='en', stop=STOP, only_standard=True))

Store

In [3]:
from os.path import join
import pandas as pd
for q in SIMPLE_QUERIES:
    df = pd.DataFrame({
        'url': pd.Series(search_results[q]),
        'program': pd.Categorical(['todo']*len(search_results[q])),
        'type': pd.Categorical(['todo']*len(search_results[q])),
        'source title': pd.Series(['todo']*len(search_results[q])),
    })
    df.to_csv(join("manual", "{name}.csv".format(name=q)))

TODO: Analysed the 10 first results manually, by reading the article and not the source/domain. labeled Pro-Control if the article was leaning toward more gun control, Anti-Control if it was supporting gun rights, Balanced if comments from both views were reflected in a balanced manner, and N/A if the article hadn't mentioned the debate. We also categorized the articles into these categories: News Article, Blog Post, Research Report, Encyclopedia Entry, Database, Shop, Advocacy Group, and List of Articles. We ignored the cards in the search results page.

In [4]:
from os.path import join
import pandas as pd

reviewed_search_results = {}
for q in SIMPLE_QUERIES:
    reviewed_search_results[q] = pd.read_csv(join("manual", "{name}.reviewed.csv".format(name=q)), usecols=['url', 'program', 'type', 'source title'])

In [5]:
import helper
helper.visualize_reviewed_search_results(reviewed_search_results)

,gun,firearm,gun control,gun rights,mass shooting
0,Wikipedia(Encyclopedia Entry),Wikipedia(Encyclopedia Entry),Wikipedia(Encyclopedia Entry),OpenSecrets.org(Advocacy Group),CNN(News Article)
1,Sportsman’s Guide(Shop),todo(todo),todo(todo),NPR(List of Articles),CNN(News Article)
2,Academy(Shop),todo(todo),todo(todo),U.S. News(News Article),Independent(News Article)
3,NBC News(News Article),todo(todo),todo(todo),Wikipedia(Encyclopedia Entry),ABC News(News Article)
4,The New York Times(News Article),todo(todo),todo(todo),The Trace(List of Articles),CBS News(News Article)
5,The Washington Post(News Article),todo(todo),todo(todo),The Oregonian(News Article),Wikipedia(Encyclopedia Entry)
6,The New York Times(News Article),todo(todo),todo(todo),Congressman Robert B. Aderholt(Blog Post),The Washington Post(News Article)
7,Pew Research Center(Research Report),todo(todo),todo(todo),The Atlantic(News Article),Mass Shootings(Database)
8,Wired(News Article),todo(todo),todo(todo),The New York Times(News Article),The Washington Post(Blog Post)


TODO: post description

### Analysing the Sources
TODO: Lippmanian device: lists of pro word and antiword per issues (ex: gun control, gun right)

In [6]:
PROCONTROL_KEYWORD = 'mass-shooting "background check"'
ANTICONTROL_KEYWORD = 'mass-shooting "gun rights"'

from googlesearch import hits, get_random_user_agent
ua = get_random_user_agent()
PROCONTROL_KEYWORD_HITS = hits(PROCONTROL_KEYWORD, tld='com', lang='en', user_agent=ua)
ANTICONTROL_KEYWORD_HITS = hits(ANTICONTROL_KEYWORD, tld='com', lang='en', user_agent=ua)

In [7]:
from urllib.parse import urlparse

source_domains = set([])
for q in SIMPLE_QUERIES:
    source_domains |= set([urlparse(url).netloc for url in reviewed_search_results[q].url])

In [8]:
source_normalized_usage_procontrol = {}
source_normalized_usage_anticontrol = {}

import time
from urllib.error import HTTPError
from googlesearch import hits, get_random_user_agent
for domain in source_domains:
    while True:
        try:
            ua = get_random_user_agent()
            procontrol_q = 'site:{domain} {keyword}'.format(domain=domain, keyword=PROCONTROL_KEYWORD)
            source_normalized_usage_procontrol[domain] = hits(
                procontrol_q, tld='com', lang='en', user_agent=ua) / PROCONTROL_KEYWORD_HITS
            time.sleep(5) # To avoid Google captcha
            anticontrol_q = 'site:{domain} {keyword}'.format(domain=domain, keyword=ANTICONTROL_KEYWORD)
            source_normalized_usage_anticontrol[domain] = hits(
                anticontrol_q, tld='com', lang='en', user_agent=ua) / ANTICONTROL_KEYWORD_HITS
            time.sleep(15) # To avoid Google captcha
            break # The infinite retry loop
        except HTTPError as e:
            if e.code != 503:
                raise e
            time.sleep(60)